# Effective Python -4. 복잡한 표현식 대신 헬퍼 함수를 작성하자 


### 헬퍼함수를 사용하지 않는 경우
    - 파이썬의 간결한 문법을 이용하면 많은 로직을 표현식 한줄로 쉽게 작성 가능하다.

In [13]:
from urllib.parse import parse_qs

my_values = parse_qs('red=5&blue=0&green=', keep_blank_values=True)
#parse_qs : 쿼리 스트링(Query String 다루기)
print(my_values)

print(repr(my_values))
#repr() vs str()
#repr : 시스템이 해당 객체를 인식할 수 있는 공식적인 문자열로 나타낼때 ㅐ사용
# str : 사용자가 보기 쉬운 형태로 보여줄 때 사용하는 것


{'red': ['5'], 'blue': ['0'], 'green': ['']}
{'red': ['5'], 'blue': ['0'], 'green': ['']}


    - 쿼리 문자열 파라미터에 따라 값이 여러개, 한개일 수 있고, 파라미터는 존재하지만 값x 혹은 파라미터 x
    - 결과 딕셔너리에 get메서드를 사용하면 각 상황에 따라 다른 값을 반환 할 것

In [9]:
print('Red:    ', my_values.get('red'))
print('Green:    ', my_values.get('green'))
print('Opacity:    ', my_values.get('opacity'))

Red:     ['5']
Green:     ['']
Opacity:     None


    - 이때 파라미터가 없거나 비어 있으면 기본값으로 0을 할당하게 해보자
    - 사용하는 트릭은 빈 문자열, 빈 리스트, 0이 모두 암시적으로 False로 평가되는 것

In [11]:
red = my_values.get('red', [''])[0] or 0
green = my_values.get('green', [''])[0] or 0
opacity = my_values.get('opacity', [''])[0] or 0
print('Red:       %r' % red)
print('Green:     %r' % green)
print('Opacitry:  %r' % opacity)

Red:       '5'
Green:     0
Opacitry:  0


    - 위 코드의 red, green, opacity 표현식에서 첫 번째 서브 표현식이 False 일때 or 연사자 뒤에 오는 서브 표현식을 평가한 값이 결과가 된다.  
        red : 키값이 my_values 딕셔너리에 존재 
               -> 값은 멤버 하나만 있는 list 
               -> 이 문자열은 True 
               -> red는 or 의 첫 번째 부분을 할당받음
        green : 키값이 my_values 딕셔너리에 존재 
               -> 빈 문자열인 list 
               -> 암시적으로 ‘False’ 
               -> or 의 결과는 0
        opacity : 키 값이 없음 
               -> get 메서드는 키가 딕셔너리에 없으면 두 번째 인수를 반환(기본값은 빈 문자열) 
               -> green과 똑같이 동작

In [15]:
red = int(my_values.get('red', [''])[0] or 0)
red

5

위식도 딱히 편하지가 않다  
if/else 조건 삼항 표현식 사용해보자 !

In [21]:
red = my_values.get('red')
print(red)
red = my_values.get('red',['']) #위것과 모가 다른거냐?
print(red)

red = int(red[0]) if red[0] else 0
print(red)

['5']
['5']
5


그래도 어렵,,,,

In [24]:
green = my_values.get('green', [''])
if green[0]:
    green = int(green[0])
else:
    green = 0
    
print(green)

0


음 ,, 보기는 좋긴한데,, 

### 헬퍼함수를 사용해보자 !!

    - 위와 같은 로직을 반복해서 써야한다면 헬퍼 함수를 만들자!

In [27]:
def get_first_int(values, key, default=0) :
    found = values.get(key, [''])
    if found[0] :
        found = int(found[0])
    else :
        found = default
    return found


green = get_first_int(my_values, 'green')
print(green)

0


    - 표현식이 복잡해지기 시작하면
        - 최대한 빨리 해당 표현식을 작은 조각으로 분할
        - 로직을 헬퍼 함수로 옮기는 방안 고려
    - 무조건 짧은 코드를 만들기 보다는 가독성을 선택하는 편이 좋다

### 핵심 정리
- 파이썬의 문법을 이용하면 한 줄짜리 표현식을 쉽게 작성할 수 있지만 코드가 복잡해지고 읽기 어려워진다
- 복잡한 표현식은 헬퍼 함수로 옮기는 게 좋다. 특히, 같은 로직을 반복해서 사용해야 한다면 헬퍼 함수를 사용하자.
- if/else 표현식을 이용하면 or나 and 같은 bool 연산자를 사용할 때보다 읽기 수월한 코드 작성 가능